In [ ]:
import time
import datetime
import numpy as np
from copy import deepcopy
import socket
import bluetooth as bt
import os
import re #Searches strings
import operator
from collections import OrderedDict #Allows the use of Ordered Dictionaries
from difflib import SequenceMatcher #Allows to quickly compare two strings to see how similar they are

#Desired energy
target_energy = 124 #Target energy in microjoules

interpolation_data = OrderedDict()
#Measured energy values at different attenuator degrees
interpolation_data[0]  = 50
interpolation_data[2]  = 60
interpolation_data[4]  = 70
interpolation_data[6]  = 80
interpolation_data[8]  = 90
interpolation_data[10] = 100
interpolation_data[12] = 120
interpolation_data[14] = 140
interpolation_data[16] = 160
interpolation_data[18] = 180
interpolation_data[20] = 200
#Power in the dragonview program at the time of measurement
interpolation_data['power'] = 1.7


def interpolate(interpolation_data, target_energy, new_power):
    """
    Linearly interpolates between the two closest datapoints depending on the new power value. 
    Requires: interpolation_data (OrderedDict), target_energy (float), new_power (float)
    """
    new_dict = OrderedDict()
    new_dict = deepcopy(interpolation_data) #Make a copy of the original measured data
    for key in new_dict.keys():
        #increase the power values by a factor of new_power/old_power
        new_dict[key] = new_dict[key] * new_power/interpolation_data['power']
    #Define efficient numpy arrays to do calculations
    attenuator = np.zeros(11,dtype = np.float32)
    new_val = np.zeros(11, dtype = np.float32)
    old_val = np.zeros(11, dtype = np.float32)
    j = 0 #counter value
    for i in range(0,22,2):
        new_val[j] = new_dict[i] #Entering the dict values in numpy array
        old_val[j] = interpolation_data[i] #Entering the dict values in the numpy array
        j += 1
    j = 0
    #Define a result numpy array to find the minimum absolute difference
    result = np.zeros(11, dtype = np.float32)
    result = np.abs(new_val - target_energy)
    res_ind = np.argmin(result) #returns the index of the minimum value
    ind1_ind2 = np.zeros(2, dtype = np.float32) #2 element array with the indexes
    #Depending on the value of the returned index determine whether it's y2 or y1 for the linear interpolation
    if new_val[res_ind] - target_energy < 0 :
        ind1_ind2[0] = res_ind
        ind1_ind2[1] = res_ind + 1
    else:
        ind1_ind2[0] = res_ind - 1
        ind1_ind2[1] = res_ind
        
    #If the required index exceeds the max, use the last two values to calculate outside the measured range (extrapolate)
    if ind1_ind2[0] >= len(result):
        ind1_ind2[1] = len(result) - 1
        ind1_ind2[0] = len(result) - 2
    if ind1_ind2[1] >= len(result):
        ind1_ind2[1] = len(result) - 1
        ind1_ind2[0] = len(result) - 2
    
    #slope for linear interpolation
    m = (new_val[int(ind1_ind2[1])] - new_val[int(ind1_ind2[0])]) / (ind1_ind2[1] - ind1_ind2[0])
    
    #The indexes are half of what they are supposed to be, therefore the result needs to be multiplied by 2
    output = ((target_energy - new_val[int(ind1_ind2[0])] + m * ind1_ind2[0]) / m) * 2
    output = round(output, 1) #attenuator value rounded to 1 decimal place
    
    return output
    
    
    
def searcher(string):
    """
    Finds any file with the .csv extension, requires a string to search
    """
    file = re.compile(r'^[^~$].+\.csv')
    return file.search(string)

def file_adder(directory):
    """
    returns a list with all the csv files in the local directory, note for windows change "/" to "\"
    """
    files =[]
    for elements in os.listdir(directory):
        if searcher(elements):
            files.append(directory+"/"+elements)
    return files

def file_get():
    """
    Opens the first .csv file on this directory
    """
    curr_dir = os.getcwd()
    target_dir = curr_dir
    csv = file_adder(target_dir)[0]
    print("Opening file: ",)
    file_handle = open(csv, "r")
    return file_handle

def line_get(file_handle):
    """
    Returns a generator that endlessly reads a file while occupying constant memory. requires file_handle(file object)
    """
    file_handle.seek(0,1)
    while True:
        line = file_handle.readline()
        if not line:
            time.sleep(10)
            continue
        yield line

def packet(obj):
    """
    Converts an obj to bytes in order to send them via TCP/IP, requires obj (any type compatible with str() function)
    """
    obj = str(obj)
    return bytes(obj, 'utf-8')

def create_client(ip_addr, port):
    """
    Creates a connection with the specified server, requires ip_addr (string) and port (integer).
    Note: server must be listening for connections
    """
    TCP_IP = ip_addr
    TCP_PORT = port
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((TCP_IP, TCP_PORT))
    return s

def process_log(line):
    """
    Processes the csv file, splitting each member into a list and removing spaces and newline characters.
    Requires line (string)
    """
    proc_data = line.split(',')
    for i in range(0, len(proc_data), 1):
        proc_data[i] = proc_data[i].strip(' ').strip('\n')
    return proc_data

def return_datetime():
    """
    Returns the date and time in the following format YYYY/MM/DD HH:MM:SS
    """
    date_time = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S')
    return date_time

def convert_to_float(string, interpolation_data):
    """
    Attempts to convert a string to a floating point, if it fails, then returns the value of power measured originally
    Requires string, interpolation_data (dictionary)
    """
    try:
        output = float(string)
        return output
    except:
        output = interpolation_data['power']
        return output

def start_monitor(ip_addr, port, inter_data, target_ener, BUFFER_SIZE = 32):
    """
    Continuously reads and transmits the attenuator values to the server
    requires:
    ip_addr (string)
    port (integer)
    inter_data (dictionary)
    target_ener (float)
    optional BUFFER_SIZE (integer)
    """
    try:
        file_handle = file_get() #Open the file
        generator = line_get(file_handle) #Create the endless line generator
        prev_power = inter_data['power'] #set the starting power as the power measured
        att_value = interpolate(inter_data, target_ener, prev_power) #set the starting att_value
        s = create_client(ip_addr, port) #Create a client TCP/IP connection
        for lines in generator: #Start getting the lines from the generator
            line = process_log(lines) #Separate the lines by commas into a list
            new_pwr = convert_to_float(line[3],interpolation_data) #convert the power value to floating point
             
            #Clause to handle the cases where the seed laser/pump laser are disconnected
            if (new_pwr != prev_power and prev_power != 0 and new_pwr != 0):
                #If the value changes more than 5% then change the attenuator val
                if np.abs(np.abs((new_pwr-prev_power)/prev_power)-1) > 0.05:
                    att_value = interpolate(inter_data, target_ener, new_pwr)
                    prev_power = new_pwr
            #Print the values read from the csv file + the att_value sent to the server and the time it was sent
            string = "log_time = {}, log_power = {} W, att_value_sent = {}, data sent {}".format(line[0], line[3], 
                                                                                        att_value, return_datetime())
            print(string, end = '\r') #'\r' means return carriage. Overwrites the same line
            s.send(packet(att_value)) #Send the att_value in bytes to the server
            data = s.recv(BUFFER_SIZE) #Waits for server response, this is necessary to prevent the client 
                        #from sending the next packet before the server is ready
    except:
        print("\nexcept clause entered: closing connections...")
        s.close() #Closes connections in the case of an exception
    
    return


    

In [ ]:
#adapted from: https://wiki.python.org/moin/TcpCommunication
#clientside

start_monitor('127.0.0.1', 5017, interpolation_data, target_energy) #Needs to be stopped manually

In [116]:
print(datetime.date.today())

2017-09-12


In [2]:
def return_test():
    try:
        a = 'aaa'
        a = float(a)
        return
    except:
        print(a)
        return

In [4]:
return_test()

aaa
